# A full business solution



### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

This notebook demonstrates how to automate the creation of a company brochure using LLMs (like GPT-4o-mini) and web scraping. 



In [23]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [24]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [25]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [26]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-wit

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.


In [27]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [28]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [29]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [30]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/
https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineerin

In [31]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [32]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/mistralai/Devstral-Small-2505',
 '/google/gemma-3n-E4B-it-litert-preview',
 '/ByteDance-Seed/BAGEL-7B-MoT',
 '/nari-labs/Dia-1.6B',
 '/google/medgemma-4b-it',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Lightricks/ltx-video-distilled',
 '/spaces/NihalGazi/FLUX-Pro-Unlimited',
 '/spaces/ByteDance/DreamO',
 '/spaces/stepfun-ai/Step1X-3D',
 '/spaces',
 '/datasets/openbmb/Ultra-FineWeb',
 '/datasets/disco-eth/EuroSpeech',
 '/datasets/ministere-culture/comparia-conversations',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/tr

In [33]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [34]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [35]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
mistralai/Devstral-Small-2505
Updated
about 16 hours ago
•
27.4k
•
463
google/gemma-3n-E4B-it-litert-preview
Updated
3 days ago
•
428
ByteDance-Seed/BAGEL-7B-MoT
Update

In [36]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [37]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [38]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'company page', 'url': 'https://huggingface.co'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmistralai/Devstral-Small-2505\nUpdated\nabout 16 hours ago\n•\n27.4k\n•\n463\ngoogle/gemma-3n-E4B-it-litert-preview\nUpdated\n3 days ago\n•\n428\nByteDance-Seed/BAGEL-7B-MoT\nUpdated\n1 day ago\n•\n784\n•\n378\nnari-labs/Dia-1.6B\nUpdated\n10 days ago\n•\n189k\n•\n2.36k\ngoogle/medgemma-4b-it\nUpdated\n2 days ago\n•\n6.03k\n•\n163\nBrowse 1M+ models\nSpaces\nRunning\n7.06k\n7.06k\nDeepSite

In [39]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [41]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

## **About Us**
Hugging Face is a vibrant AI community dedicated to building the future through collaboration in machine learning. We are not just a platform; we are the heart of innovation in AI, where individuals and organizations come together to create, discover, and advance state-of-the-art machine learning technologies. 

## **Our Platform**
Hugging Face hosts an extensive collection of tools and resources, including:
- **1M+ Models:** Discover and utilize a diverse range of pre-trained models for various applications.
- **250k+ Datasets:** Access and share datasets for any machine learning tasks.
- **400k+ Applications:** Create and run machine learning applications through our unique Spaces feature.

### **What We Offer**
- **Open Source Stack:** We provide cutting-edge ML resources, including Transformers for PyTorch, TensorFlow, and JAX, making it easier to innovate and experiment with various modalities such as text, image, video, audio, and 3D.
- **Enterprise Solutions:** Our enterprise platform features include dedicated support, access controls, and security, ensuring that organizations can scale and secure their machine learning efforts effectively.
- **Compute & Pricing:** Starting from $0.60/hour for GPU deployments, we offer flexible and competitive pricing to meet any project needs.

## **Our Community**
Over **50,000 organizations** leverage our resources, including leading companies like:
- **Meta**
- **Amazon**
- **Google**
- **Microsoft**

Our mission is to democratize machine learning by fostering an inclusive, supportive, and collaborative environment that inspires innovation and knowledge sharing.

## **Company Culture**
At Hugging Face, we pride ourselves on our open, inclusive, and transparent culture. Our team thrives on collaboration and diversity, encouraging every member to contribute their unique perspectives and ideas. We believe in the power of community, as reflected in our mission to democratize machine learning one commit at a time. 

## **Careers at Hugging Face**
We are always on the lookout for talented individuals who share our passion for machine learning and community building. If you're excited about making a difference in the AI landscape and want to work with a dedicated and dynamic team, we'd love to hear from you! 

[**Explore Careers**](https://huggingface.co/jobs)

## **Join Us!**
Become a part of the Hugging Face community and help shape the future of AI. Whether you're a researcher, developer, or enthusiast, there's a place for you here!

[**Sign Up Now!**](https://huggingface.co/signup)

## **Connect with Us**
Stay connected and follow our journey on social media!
- **GitHub**
- **Twitter**
- **LinkedIn**
- **Discord**

---

For additional information, visit our [Website](https://huggingface.co) and explore all we have to offer in the exciting world of machine learning!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [42]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [43]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'social media LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'social media Twitter', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Company Brochure

## Welcome to Hugging Face
At Hugging Face, we are the AI community dedicated to building the future. Our platform is designed for collaboration, supporting the machine learning community in developing models, datasets, and applications to make AI accessible to all.

### Our Offerings
- **Models**: Explore over 1 million models across various domains.
- **Datasets**: Access and share over 250,000 datasets for any machine learning task.
- **Spaces**: Leverage our platform to create, run, and collaborate on applications using powerful AI technologies.

### Community Engagement
Hugging Face is more than just a platform; it's a thriving community of over 50,000 organizations, including entities such as Google, Microsoft, and Amazon. Our users can share their work, collaborate on projects, and build a robust portfolio while contributing to the advancement of AI technologies.

### Company Culture
At Hugging Face, our culture thrives on innovation, collaboration, and a deep commitment to the open-source philosophy. We believe in building tools that are not only powerful but also accessible. Our platform encourages community engagement, and we value contributions from all members, creating an inclusive atmosphere for learning and growth.

### Enterprise Solutions
For businesses looking for advanced tools, our enterprise offerings provide top-grade security, dedicated support, and the performance required for mission-critical applications. Starting from $20/user/month, organizations can benefit from our enterprise-grade features, ensuring their teams have the right resources to succeed.

### Careers at Hugging Face
We are always looking for passionate individuals to join our growing team. With a focus on artificial intelligence, machine learning, and open-source technologies, careers at Hugging Face are rewarding and impactful. We value creativity, drive, and the ability to work collaboratively in a fast-paced environment.

### Join Us
Explore our platform, contribute to the community, and discover how you can help shape the future of AI. Whether you're a developer, researcher, or a business leader, there's a place for you at Hugging Face.

**Learn more about our offerings, career opportunities, and community initiatives at [Hugging Face](https://huggingface.co).**

---

**Contact Us:**
- **Website**: [huggingface.co](https://huggingface.co)
- **Email**: support@huggingface.co

Let’s build the future of AI together!

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'company page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face: The AI Community Building the Future

## About Us
Welcome to Hugging Face, a collaborative platform for the machine learning community where innovation thrives. With over 1 million models and 250,000 datasets, we empower developers, researchers, and enterprises to create and share advanced AI applications.

### Our Mission
At Hugging Face, we are dedicated to democratizing AI through open source solutions. We believe in building the future of machine learning by fostering collaboration among individuals and organizations alike. 

## Our Offerings
- **Models**: Browse and utilize over 1 million machine learning models.
- **Datasets**: Access 250,000+ datasets to enhance your projects.
- **Spaces**: Create and share applications effortlessly on our user-friendly platform.

## Customer Base
Join more than 50,000 organizations leveraging our robust tools, including major tech giants like Google, Microsoft, Amazon, and more. Our platform is trusted by diverse users ranging from non-profits to enterprise-level companies.

## Community
Hugging Face thrives on community engagement. We provide resources to help users share their findings, improve their portfolios, and collaborate on transformative AI applications. Whether you’re a beginner or an expert, you’ll find a welcoming environment here.

## Careers at Hugging Face
We are always looking for passionate individuals to join our team. If you aspire to be part of a team that values innovation, collaboration, and support, consider applying for a position with us. 

- **Why Work With Us?**
  - **Inclusive Culture**: We foster a diverse work environment where every perspective is valued.
  - **Growth Opportunities**: We promote continuous learning and professional development.
  - **Flexibility**: Embrace a work-life balance that suits your lifestyle.

### Current Job Openings
Visit our [Jobs page](#) to explore available positions and become a part of our mission to enhance AI accessibility for everyone.

## Join Us
Together, let’s shape the future of AI! Sign up today and start building with us.

[Sign Up Now](#)

For more insights, check out our documentation, blog, and community discussions. Follow us on [Twitter](#), [LinkedIn](#), and [Discord](#) for updates and support.

---

**Hugging Face – Where the machine learning community collaborates on models, datasets, and applications.**


In [45]:
create_brochure("LSU", "https://www.lsu.edu/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.lsu.edu/about/index.php'}, {'type': 'admissions page', 'url': 'https://www.lsu.edu/admission/index.php'}, {'type': 'careers page', 'url': 'https://lsu.edu/hrm/careers/index.php'}, {'type': 'campus information page', 'url': 'https://www.lsu.edu/campuses.php'}, {'type': 'academic offerings page', 'url': 'https://www.lsu.edu/majors/index.php'}, {'type': 'student affairs page', 'url': 'https://www.lsu.edu/student-affairs/index.php'}, {'type': 'financial aid page', 'url': 'https://www.lsu.edu/financialaid/index.php'}, {'type': 'graduate school page', 'url': 'https://www.lsu.edu/graduateschool/index.php'}, {'type': 'research page', 'url': 'https://www.lsu.edu/research/index.php'}, {'type': 'continuing education page', 'url': 'https://online.lsu.edu/continuing-education/index.php'}]}


```markdown
# Welcome to LSU

**Louisiana State University** is a top-tier research university located in **Baton Rouge, Louisiana**. Known for its vibrant community and commitment to academic excellence, LSU offers a diverse range of programs designed to equip students for success in their individual fields.

## Our Mission

At LSU, we are dedicated to changing lives through education, discovery, collaboration, and leadership. We provide our students with the resources they need to explore their ambitions, pursue their dreams, and elevate their opportunities.

## Company Culture

As a leading research institution, LSU fosters a collaborative and inclusive environment. Our community is enriched by diverse perspectives and talents. We empower our students, faculty, and staff to embrace challenges and contribute positively to society. Whether it's through academics, athletics, or extracurricular activities, we believe in the success of every individual who becomes part of our Tiger family.

## Academics

- **Undergraduate Programs:** Offering over 70 master's and 50 doctoral degree paths, LSU provides a wide array of academic opportunities, including highly-ranked programs in agriculture, business, engineering, and the health sciences.
- **Graduate Opportunities:** LSU's graduate school ensures advanced education through interdisciplinary approaches, dual degrees, and even online options for remote learners.
- **Research Initiatives:** Our research endeavors tackle real-world challenges ranging from cancer research to energy innovation, with an emphasis on sustainable practices and economic development.

## Community Engagement

LSU prides itself on being engaged with the community. Our initiatives not only benefit our students but also enhance the local region. Partnerships with industry and community organizations help us extend our impact beyond the campus.

## Careers at LSU

Joining LSU means becoming part of an elite and historic institution. We are committed to fostering talent and providing our employees with professional growth opportunities. With various roles ranging from academic faculty to administrative positions, there’s a place for everyone. 

**Explore Career Opportunities:**
- Tap into various departments, from student affairs to research and development.
- Engage with a vibrant network of colleagues dedicated to excellence.

## Join Us!

Whether you're a prospective student eager to learn, a potential investor looking for collaboration, or a recruit seeking a meaningful career, LSU offers something for everyone. 

**Become a Tiger today!** 
- Apply now or visit our campus to learn more about what LSU can offer you.
- For more information, visit [LSU.edu](https://www.lsu.edu).

Let’s achieve greatness together!
```


In [46]:
stream_brochure("LSU", "https://www.lsu.edu/")

Found links: {'links': [{'type': 'about page', 'url': 'https://lsu.edu/about/index.php'}, {'type': 'careers page', 'url': 'https://lsu.edu/hrm/careers/index.php'}, {'type': 'admissions page', 'url': 'https://admissions.lsu.edu/register/?id=51584c6f-5a25-44d1-9877-a0ce21309fd9'}, {'type': 'scholarship page', 'url': 'https://lsu.edu/scholarship-first/index.php'}, {'type': "president's page", 'url': 'https://lsu.edu/president/index.php'}, {'type': 'campuses page', 'url': 'https://lsu.edu/campuses.php'}, {'type': 'student life page', 'url': 'https://lsu.edu/student-affairs/student-life/index.php'}]}



# Welcome to LSU

**Louisiana State University**  
*Baton Rouge, Louisiana*  

---

## Who We Are

LSU is a leading research university renowned for its strong academic programs and vibrant community. With a dedication to education, discovery, and collaboration, we empower students, faculty, and alumni to make significant contributions to society. LSU serves as Louisiana's flagship university, fostering an environment where academic excellence and teamwork flourish.

---

## Our Academic Programs

LSU offers a wide array of academic opportunities with over 70 master’s and 50 doctoral degree programs spanning various disciplines. Our colleges and schools include:

- College of Agriculture
- College of Art & Design
- E. J. Ourso College of Business
- College of Coast & Environment
- College of Human Sciences & Education
- College of Humanities & Social Sciences
- Manship School of Mass Communication
- College of Music & Dramatic Arts
- College of Engineering
- School of Veterinary Medicine
- LSU Paul M. Hebert Law Center
- Pinkie Gordon Lane Graduate School
- College of Science

We emphasize research that drives impactful discoveries, addressing challenges in areas like healthcare, sustainability, and technology.

---

## Our Community

At LSU, we believe in a united and inclusive community. We provide various student resources including counseling, health services, dining, and opportunities for involvement through organizations, Greek life, and student government. You can also participate in campus events and initiatives that encourage student engagement and success.

**Join a Vibrant Community**  
Become part of a diverse group of students and scholars committed to personal and academic growth. Our campus life is alive with energy, fostering friendships, collaboration, and fun.

---

## Careers at LSU

Working at LSU means being part of a team that values innovation, accountability, and achievement. Faculty and staff enjoy the chance to engage in impactful work, contribute to educational excellence, and further their professional development.

**Join Our Team!**  
Explore career opportunities at LSU where you can play a role in shaping future leaders and contributing positively to the community.

---

## Why Choose LSU?

- **Accredited Programs**: Nationally recognized degrees across various fields.
- **Campus Safety**: Ensuring a safe environment for all students and staff.
- **Support**: Comprehensive financial aid and counseling services are available to help you succeed.
- **Online Learning**: Flexible options for earning your degree, including fully online programs.

---

## Connect With Us

**Ready to Dive into Greatness?**  
Explore the many opportunities waiting for you at LSU.  
[Apply Today](#) or [Plan a Visit](#) to experience our vibrant campus and community!  
Together, we can achieve greatness and change the world.

---

*We Build Teams That Win: Changing Lives Through Education, Discovery, and Leadership.*

For more information, visit [our website](https://www.lsu.edu/).

